In [161]:
import pandas as pd

In [195]:
"""
Input: 
    Text file containing all the generated fake reviews from GPT2 fine tuned model
Output:
    Saves a 'fake_reviews.csv' in the current directory that labels every review as fake
    
Fake reviews are filtered to remove \n symbol and the GTP2 model 
included some lines containing only equal signs which are also removed.
"""
def create_df_from_txt(path):
    
    review_list = []
    substr = "======"
    with open("gpt_2_gen_texts.txt", 'r') as generated_reviews:
        for line in generated_reviews:
            #append all lines that are not equal signs
            if substr not in line:
                #remove newline token with [:-1]
                review_list.append(line[:-1])
                
    df = pd.DataFrame(review_list, columns=['Review'])
    df['Real'] = 0
    df.dropna(inplace=True)
    
    df.to_csv("fake_reviews.csv", index=False)
    print("Saved fake_reviews.csv")
    return df

In [196]:
generate_reviews_path = "gpt_2_gen_texts.txt"
d = create_df_from_txt(generate_reviews_path)

Saved fake_reviews.csv


In [197]:
real_reviews = pd.read_csv("real_reviews.csv")
fake_reviews = pd.read_csv("fake_reviews.csv")

In [198]:
#50% real 50% fake in order to avoid biased data
real_reviews = real_reviews.iloc[0:fake_reviews.shape[0]]
assert(real_reviews.shape == fake_reviews.shape)

In [202]:
merged_df = pd.concat([fake_reviews, real_reviews], ignore_index=True)
merged_df.to_csv("classifier_data.csv", index=False)